In [1]:
%run Regard.py --no-cuda

cuda? False


AttributeError: 'torch.LongTensor' object has no attribute 'requires_grad'

Process Process-2:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process Process-1:
  File "/home/nicolas/.local/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    sel

In [40]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import matplotlib.pyplot as plt
import torchvision


# Training settings
cuda = False
torch.manual_seed(42)

kwargs = {'num_workers': 1, 'pin_memory': True}

dataset = torchvision.datasets.ImageFolder('dataset',
                                        transforms.Compose([
                                        transforms.CenterCrop(100),
                                        transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                        ])) 
train_loader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=2)
test_loader= torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=2)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(500, args.dimension)# args.dimension prend la valeur default de l'argument dimension.
        self.fc2 = nn.Linear(args.dimension, 4)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 4))        
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 4))       
        x = x.view(-1, 500)        
        x = F.relu(self.fc1(x))        
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)        
        return F.log_softmax(x, dim=1)

model = Net()

if cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()#.cuda()#
criterion = F.nll_loss
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):#,0):
        print(40*'-', batch_idx)
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target) #[batch_idx].unsqueeze(0)
        print(data.size(), target.size())
              
        optimizer.zero_grad()
        output = model(data)#
        output_max=torch.max(output,1)
        print('data=',data.shape)
        print ('output_max=',output_max)
        
        #print ('indent=',indent,type(indent))
        print ('target=',target,type(target))
        #loss = F.nll_loss(output, target)
        
        loss = criterion(output, target-1)#
        loss.backward()
        optimizer.step()
        if args.log_interval>0: # rajout de la commande pour pouvoir print ou non les différents epoch ou juste le résultat
            if batch_idx % args.log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0]))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    if args.log_interval>0: print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return correct / len(test_loader.dataset)

def protocol():
    for epoch in range(10):
        train(epoch)
    return test()

def main():
    for epoch in range(10):
        train(epoch)
        Accuracy = test()
    print('Test set: Final Accuracy: {:.3f}%'.format(Accuracy*100)) # print que le pourcentage de réussite final
    
    
t0 = time.time () # ajout de la constante de temps t0

main()

t1 = time.time () # ajout de la constante de temps t1

print ("Le programme a mis",t1-t0, "secondes à s'exécuter.") #compare t1 et t0, connaitre le temps d'execution du programme

---------------------------------------- 0
torch.Size([10, 3, 100, 100]) torch.Size([10])
x= torch.Size([10, 3, 100, 100])
x1= torch.Size([10, 10, 24, 24])
x2= torch.Size([10, 20, 5, 5])
x3= torch.Size([10, 500])
x4= torch.Size([10, 120])
x5= torch.Size([10, 120])
x6= torch.Size([10, 4])
data= torch.Size([10, 3, 100, 100])
output_max= (Variable containing:
-1.2387
-1.2835
-1.2405
-1.3448
-1.1073
-1.1072
-1.2665
-1.2401
-0.8644
-1.0721
[torch.FloatTensor of size 10]
, Variable containing:
 0
 0
 2
 1
 1
 1
 1
 1
 1
 1
[torch.LongTensor of size 10]
)
target= Variable containing:
 2
 2
 2
 2
 2
 2
 1
 2
 1
 2
[torch.LongTensor of size 10]
 <class 'torch.autograd.variable.Variable'>
Train Epoch: 0 [0/393 (0%)]	Loss: 1.296394
---------------------------------------- 1
torch.Size([10, 3, 100, 100]) torch.Size([10])
x= torch.Size([10, 3, 100, 100])
x1= torch.Size([10, 10, 24, 24])
x2= torch.Size([10, 20, 5, 5])
x3= torch.Size([10, 500])
x4= torch.Size([10, 120])
x5= torch.Size([10, 120])
x6= 

---------------------------------------- 17
torch.Size([10, 3, 100, 100]) torch.Size([10])
x= torch.Size([10, 3, 100, 100])
x1= torch.Size([10, 10, 24, 24])
x2= torch.Size([10, 20, 5, 5])
x3= torch.Size([10, 500])
x4= torch.Size([10, 120])
x5= torch.Size([10, 120])
x6= torch.Size([10, 4])
data= torch.Size([10, 3, 100, 100])
output_max= (Variable containing:
-1.1249
-1.2696
-1.2978
-1.3368
-1.1160
-1.2593
-1.2654
-1.1991
-1.2219
-1.3313
[torch.FloatTensor of size 10]
, Variable containing:
 1
 1
 3
 2
 1
 3
 2
 1
 1
 1
[torch.LongTensor of size 10]
)
target= Variable containing:
 4
 3
 4
 3
 2
 2
 2
 4
 3
 2
[torch.LongTensor of size 10]
 <class 'torch.autograd.variable.Variable'>
---------------------------------------- 18
torch.Size([10, 3, 100, 100]) torch.Size([10])
x= torch.Size([10, 3, 100, 100])
x1= torch.Size([10, 10, 24, 24])
x2= torch.Size([10, 20, 5, 5])
x3= torch.Size([10, 500])
x4= torch.Size([10, 120])
x5= torch.Size([10, 120])
x6= torch.Size([10, 4])
data= torch.Size([10,

---------------------------------------- 29
torch.Size([10, 3, 100, 100]) torch.Size([10])
x= torch.Size([10, 3, 100, 100])
x1= torch.Size([10, 10, 24, 24])
x2= torch.Size([10, 20, 5, 5])
x3= torch.Size([10, 500])
x4= torch.Size([10, 120])
x5= torch.Size([10, 120])
x6= torch.Size([10, 4])
data= torch.Size([10, 3, 100, 100])
output_max= (Variable containing:
-0.8569
-0.7654
-1.0818
-1.0878
-1.2012
-1.1633
-0.7131
-0.5581
-1.2141
-1.2035
[torch.FloatTensor of size 10]
, Variable containing:
 3
 3
 3
 3
 2
 2
 3
 3
 1
 2
[torch.LongTensor of size 10]
)
target= Variable containing:
 2
 2
 4
 2
 2
 4
 4
 3
 4
 3
[torch.LongTensor of size 10]
 <class 'torch.autograd.variable.Variable'>
---------------------------------------- 30
torch.Size([10, 3, 100, 100]) torch.Size([10])
x= torch.Size([10, 3, 100, 100])
x1= torch.Size([10, 10, 24, 24])
x2= torch.Size([10, 20, 5, 5])
x3= torch.Size([10, 500])
x4= torch.Size([10, 120])
x5= torch.Size([10, 120])
x6= torch.Size([10, 4])
data= torch.Size([10,

RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at /pytorch/torch/lib/THNN/generic/ClassNLLCriterion.c:87

In [33]:
for batch_idx, (data, target) in enumerate(train_loader):#,0):
    if args.cuda:
        data, target = data.cuda(), target.cuda()
    data, target = Variable(data), Variable(target) #[batch_idx].unsqueeze(0)
    print(data.size(), target.size())

    optimizer.zero_grad()
    output = model(data)#
    output_max=torch.max(output,1)
    print('data=',data.shape)
    print ('output_max=',output_max)

    #print ('indent=',indent,type(indent))
    print ('target=',target,type(target))
    #loss = F.nll_loss(output, target)


torch.Size([10, 3, 100, 100]) torch.Size([10])
x= torch.Size([10, 3, 100, 100])
x1= torch.Size([10, 10, 24, 24])
x2= torch.Size([10, 20, 5, 5])
x3= torch.Size([10, 500])
x4= torch.Size([10, 120])
x5= torch.Size([10, 120])
x6= torch.Size([10, 4])
data= torch.Size([10, 3, 100, 100])
output_max= (Variable containing:
-1.0053
-1.1620
-0.9536
-1.1968
-1.1602
-1.1837
-1.2002
-1.3235
-0.9458
-0.9863
[torch.FloatTensor of size 10]
, Variable containing:
 1
 1
 1
 1
 2
 1
 2
 0
 1
 1
[torch.LongTensor of size 10]
)
target= Variable containing:
 3
 3
 3
 2
 1
 2
 3
 3
 2
 4
[torch.LongTensor of size 10]
 <class 'torch.autograd.variable.Variable'>
torch.Size([10, 3, 100, 100]) torch.Size([10])
x= torch.Size([10, 3, 100, 100])
x1= torch.Size([10, 10, 24, 24])
x2= torch.Size([10, 20, 5, 5])
x3= torch.Size([10, 500])
x4= torch.Size([10, 120])
x5= torch.Size([10, 120])
x6= torch.Size([10, 4])
data= torch.Size([10, 3, 100, 100])
output_max= (Variable containing:
-1.3216
-1.2141
-1.1857
-0.9686
-1.0989

torch.Size([10, 3, 100, 100]) torch.Size([10])
x= torch.Size([10, 3, 100, 100])
x1= torch.Size([10, 10, 24, 24])
x2= torch.Size([10, 20, 5, 5])
x3= torch.Size([10, 500])
x4= torch.Size([10, 120])
x5= torch.Size([10, 120])
x6= torch.Size([10, 4])
data= torch.Size([10, 3, 100, 100])
output_max= (Variable containing:
-1.1997
-1.1346
-1.2410
-0.9823
-1.0360
-1.2474
-1.2536
-1.2802
-1.1997
-1.3465
[torch.FloatTensor of size 10]
, Variable containing:
 1
 2
 1
 1
 2
 0
 1
 1
 1
 1
[torch.LongTensor of size 10]
)
target= Variable containing:
 4
 2
 3
 4
 2
 4
 4
 3
 3
 4
[torch.LongTensor of size 10]
 <class 'torch.autograd.variable.Variable'>
torch.Size([10, 3, 100, 100]) torch.Size([10])
x= torch.Size([10, 3, 100, 100])
x1= torch.Size([10, 10, 24, 24])
x2= torch.Size([10, 20, 5, 5])
x3= torch.Size([10, 500])
x4= torch.Size([10, 120])
x5= torch.Size([10, 120])
x6= torch.Size([10, 4])
data= torch.Size([10, 3, 100, 100])
output_max= (Variable containing:
-1.2831
-1.3392
-1.1085
-1.1872
-1.1944

In [34]:
output, output_max

(Variable containing:
 -1.4299 -1.2185 -1.3794 -1.5452
 -1.3590 -1.3485 -1.4205 -1.4195
 -1.8727 -0.8171 -1.2753 -2.0776
 [torch.FloatTensor of size 3x4], (Variable containing:
  -1.2185
  -1.3485
  -0.8171
  [torch.FloatTensor of size 3], Variable containing:
   1
   1
   1
  [torch.LongTensor of size 3]))

In [35]:
output_max

(Variable containing:
 -1.2185
 -1.3485
 -0.8171
 [torch.FloatTensor of size 3], Variable containing:
  1
  1
  1
 [torch.LongTensor of size 3])

In [36]:
loss = criterion(output, target)

RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at /pytorch/torch/lib/THNN/generic/ClassNLLCriterion.c:87

In [38]:
target

Variable containing:
 3
 2
 4
[torch.LongTensor of size 3]

In [39]:
target -1

Variable containing:
 2
 1
 3
[torch.LongTensor of size 3]

In [ ]:
% run mnist.py --log-interval 0 --epochs 1

In [29]:
from __future__ import print_function
import argparse
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings

print ('cuda?', torch.cuda.is_available())
torch.manual_seed(44)
kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=10, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=10, shuffle=True, **kwargs)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)# args.dimension prend la valeur default de l'argument dimension.
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net()
if args.cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if args.log_interval>0: # rajout de la commande pour pouvoir print ou non les différents epoch ou juste le résultat
            if batch_idx % args.log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    if args.log_interval>0: print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return correct / len(test_loader.dataset)

def protocol():
    for epoch in range(1, args.epochs + 1):
        train(epoch)
    return test()

def main():
    for epoch in range(1, args.epochs + 1):
        train(epoch)
        Accuracy = test()
    print('Test set: Final Accuracy: {:.3f}%'.format(Accuracy*100)) # print que le pourcentage de réussite final
    
    

cuda? True


In [25]:
main()

cuda? True
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.415856
Train Epoch: 1 [100/60000 (0%)]	Loss: 2.289357
Train Epoch: 1 [200/60000 (0%)]	Loss: 2.287842
Train Epoch: 1 [300/60000 (0%)]	Loss: 2.274595
Train Epoch: 1 [400/60000 (1%)]	Loss: 2.287816
Train Epoch: 1 [500/60000 (1%)]	Loss: 2.305919
Train Epoch: 1 [600/60000 (1%)]	Loss: 2.238723
Train Epoch: 1 [700/60000 (1%)]	Loss: 2.199195
Train Epoch: 1 [800/60000 (1%)]	Loss: 2.143682
Train Epoch: 1 [900/60000 (2%)]	Loss: 2.180805
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.194519
Train Epoch: 1 [1100/60000 (2%)]	Loss: 2.168604
Train Epoch: 1 [1200/60000 (2%)]	Loss: 1.938203
Train Epoch: 1 [1300/60000 (2%)]	Loss: 1.988117
Train Epoch: 1 [1400/60000 (2%)]	Loss: 1.586117
Train Epoch: 1 [1500/60000 (2%)]	Loss: 1.949722
Train Epoch: 1 [1600/60000 (3%)]	Loss: 1.819917
Train Epoch: 1 [1700/60000 (3%)]	Loss: 1.387675
Train Epoch: 1 [1800/60000 (3%)]	Loss: 1.617591
Train Epoch: 1 [1900/60000 (3%)]	Loss: 1.592736
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.

Train Epoch: 1 [16800/60000 (28%)]	Loss: 0.413692
Train Epoch: 1 [16900/60000 (28%)]	Loss: 0.351050
Train Epoch: 1 [17000/60000 (28%)]	Loss: 0.457967
Train Epoch: 1 [17100/60000 (28%)]	Loss: 0.267647
Train Epoch: 1 [17200/60000 (29%)]	Loss: 0.635248
Train Epoch: 1 [17300/60000 (29%)]	Loss: 0.215838
Train Epoch: 1 [17400/60000 (29%)]	Loss: 0.544601
Train Epoch: 1 [17500/60000 (29%)]	Loss: 1.269762
Train Epoch: 1 [17600/60000 (29%)]	Loss: 0.923326
Train Epoch: 1 [17700/60000 (30%)]	Loss: 0.323928
Train Epoch: 1 [17800/60000 (30%)]	Loss: 0.683488
Train Epoch: 1 [17900/60000 (30%)]	Loss: 0.215470
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.419810
Train Epoch: 1 [18100/60000 (30%)]	Loss: 0.992294
Train Epoch: 1 [18200/60000 (30%)]	Loss: 0.414186
Train Epoch: 1 [18300/60000 (30%)]	Loss: 0.536497
Train Epoch: 1 [18400/60000 (31%)]	Loss: 0.557489
Train Epoch: 1 [18500/60000 (31%)]	Loss: 0.550288
Train Epoch: 1 [18600/60000 (31%)]	Loss: 0.148705
Train Epoch: 1 [18700/60000 (31%)]	Loss: 0.258532


Train Epoch: 1 [33400/60000 (56%)]	Loss: 0.558988
Train Epoch: 1 [33500/60000 (56%)]	Loss: 0.338025
Train Epoch: 1 [33600/60000 (56%)]	Loss: 0.111410
Train Epoch: 1 [33700/60000 (56%)]	Loss: 0.435465
Train Epoch: 1 [33800/60000 (56%)]	Loss: 0.232311
Train Epoch: 1 [33900/60000 (56%)]	Loss: 0.323663
Train Epoch: 1 [34000/60000 (57%)]	Loss: 0.132248
Train Epoch: 1 [34100/60000 (57%)]	Loss: 0.172165
Train Epoch: 1 [34200/60000 (57%)]	Loss: 0.252455
Train Epoch: 1 [34300/60000 (57%)]	Loss: 0.372268
Train Epoch: 1 [34400/60000 (57%)]	Loss: 0.486066
Train Epoch: 1 [34500/60000 (58%)]	Loss: 0.428754
Train Epoch: 1 [34600/60000 (58%)]	Loss: 0.432736
Train Epoch: 1 [34700/60000 (58%)]	Loss: 0.170710
Train Epoch: 1 [34800/60000 (58%)]	Loss: 0.465786
Train Epoch: 1 [34900/60000 (58%)]	Loss: 0.629376
Train Epoch: 1 [35000/60000 (58%)]	Loss: 0.169195
Train Epoch: 1 [35100/60000 (58%)]	Loss: 0.267520
Train Epoch: 1 [35200/60000 (59%)]	Loss: 0.332796
Train Epoch: 1 [35300/60000 (59%)]	Loss: 0.552678


Train Epoch: 1 [49900/60000 (83%)]	Loss: 0.322733
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.020118
Train Epoch: 1 [50100/60000 (84%)]	Loss: 0.036235
Train Epoch: 1 [50200/60000 (84%)]	Loss: 0.115878
Train Epoch: 1 [50300/60000 (84%)]	Loss: 0.045836
Train Epoch: 1 [50400/60000 (84%)]	Loss: 0.141155
Train Epoch: 1 [50500/60000 (84%)]	Loss: 0.034315
Train Epoch: 1 [50600/60000 (84%)]	Loss: 0.394361
Train Epoch: 1 [50700/60000 (84%)]	Loss: 0.647950
Train Epoch: 1 [50800/60000 (85%)]	Loss: 0.156849
Train Epoch: 1 [50900/60000 (85%)]	Loss: 0.159053
Train Epoch: 1 [51000/60000 (85%)]	Loss: 0.124836
Train Epoch: 1 [51100/60000 (85%)]	Loss: 0.057172
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.459243
Train Epoch: 1 [51300/60000 (86%)]	Loss: 0.135976
Train Epoch: 1 [51400/60000 (86%)]	Loss: 0.111017
Train Epoch: 1 [51500/60000 (86%)]	Loss: 0.186561
Train Epoch: 1 [51600/60000 (86%)]	Loss: 0.155785
Train Epoch: 1 [51700/60000 (86%)]	Loss: 0.365653
Train Epoch: 1 [51800/60000 (86%)]	Loss: 0.285476


Train Epoch: 2 [6700/60000 (11%)]	Loss: 0.142540
Train Epoch: 2 [6800/60000 (11%)]	Loss: 0.973073
Train Epoch: 2 [6900/60000 (12%)]	Loss: 0.212491
Train Epoch: 2 [7000/60000 (12%)]	Loss: 0.032405
Train Epoch: 2 [7100/60000 (12%)]	Loss: 0.161163
Train Epoch: 2 [7200/60000 (12%)]	Loss: 0.737788
Train Epoch: 2 [7300/60000 (12%)]	Loss: 1.208213
Train Epoch: 2 [7400/60000 (12%)]	Loss: 0.347416
Train Epoch: 2 [7500/60000 (12%)]	Loss: 0.529697
Train Epoch: 2 [7600/60000 (13%)]	Loss: 0.077541
Train Epoch: 2 [7700/60000 (13%)]	Loss: 0.210881
Train Epoch: 2 [7800/60000 (13%)]	Loss: 0.062136
Train Epoch: 2 [7900/60000 (13%)]	Loss: 0.087072
Train Epoch: 2 [8000/60000 (13%)]	Loss: 0.349041
Train Epoch: 2 [8100/60000 (14%)]	Loss: 0.057755
Train Epoch: 2 [8200/60000 (14%)]	Loss: 0.542485
Train Epoch: 2 [8300/60000 (14%)]	Loss: 0.053166
Train Epoch: 2 [8400/60000 (14%)]	Loss: 0.306152
Train Epoch: 2 [8500/60000 (14%)]	Loss: 1.505041
Train Epoch: 2 [8600/60000 (14%)]	Loss: 0.190956
Train Epoch: 2 [8700

...

Process Process-39:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/nicolas/.local/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()


Train Epoch: 3 [30600/60000 (51%)]	Loss: 0.084179
Train Epoch: 3 [30700/60000 (51%)]	Loss: 0.095427
Train Epoch: 3 [30800/60000 (51%)]	Loss: 0.458228
Train Epoch: 3 [30900/60000 (52%)]	Loss: 0.070295


  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)


RuntimeError: DataLoader worker (pid 12714) exited unexpectedly with exit code 1.

KeyboardInterrupt


In [30]:

model.train()
for batch_idx, (data, target) in enumerate(train_loader):
    if args.cuda:
        data, target = data.cuda(), target.cuda()
    data, target = Variable(data), Variable(target)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()

In [31]:
target

Variable containing:
 2
 6
 8
 0
 5
 0
 0
 1
 5
 8
[torch.LongTensor of size 10]